# Trabalho prático II - Classificação

O objetivo deste trabalho é praticar os conceitos de aprendizado supervisionado que vimos em sala.

A sua tarefa será treinar um classificador para um conjunto de dados misterioso (se eu falasses qual é o objetivo do modelo, você encontraria soluções na Internet).

Baixe o arquivo [t2.tar.gz](https://drive.google.com/file/d/125plHKUzFGxHjjCiVJcTQG2bPG_zgDNV/view?usp=sharing). Descoprima este arquivo para encontrar outros quatro. Os arquivos `train_X.csv` e `train_y.csv` possuem os dados que você deve usar para treinar o modelo. O arquivo `test_X.csv` possui os objetos para os quais você deve encontrar as classes (testar o modelo). Por fim, o arquivo `test_example_y.csv` é um exemplo de como sua solução final deve ser organizada.

Cada linha dos arquivos `train_X.csv` e `test_X.csv` tem 15 campos descrevendo um objeto misterioso. O campo `id` representa o identificador do objeto, sendo que este campo não deve ser considerado em seu modelo. Os atributos a serem usados no modelo são os 14 campos rotulados de de `a` até `n`. Desses atributos:
- `b`, `d`, `f`, `g`, `h`, `i`, `j` e `n` são categóricos; e
- `a`, `c`, `e`, `k`, `l` e `m` são numéricos.

Cada linha do arquivo `train_y.csv` possui dois campos. O primeiro é o identificador de um objeto do arquivo `train_X.csv` e o segundo é a classe do respectivo objeto (0 ou 1).

Seu objetivo é encontrar as classes dos objetos do arquivo `test_X.csv` e mostrar como chegou em sua solução! Os dados do arquivo de teste foram obtido a partir de uma amostra aleatório do todo. Ou seja, um modelo bem treinado, e sem _overfitting_, em `train_X.csv` e `train_y.csv` se sairá bem em `test_X.csv`.

**Data de entrega:** dia 4 de julho de 2018.

**Grupo:** de até 3 pessoas, mas duas pessoas do mesmo grupo no trabalho 1 não podem pertencer ao mesmo grupo nesse trabalho.

**Valor:** 20% da nota do semestre.

Os três seguintes pontos descrevem o que obrigatoriamente deve ser entregue, com seu respectivo valor.

1 - **[10 pontos]** Este notebook com todo seu código e resultados (números, tabelas e gráficos). Você pode usar qualquer um dos métodos que estudamos ou alguma de suas variações próximas. Se estiver na dúvida se pode usar um método, basta perguntas no Piazza. Comentários e justificativas no notebook não serão considerados para sua nota.
O notebook deve ser enviado para o email do professor.

2 - **[8 pontos]** Um relatório digitado contendo: capa, introdução, metodologia, resultados, conclusão e referências. O relatório deve ter no máximo 10 páginas, com coluna simples, fonte 11, espaçamento 1.5 e margens de 2cm. A seção de metodologia deve conter uma descrição detalhada dos passos seguidos (não incluir código no relatório). A seção de resultados deve conter obrigatoriamente: uma caracterização descritiva dos dados, matriz de confusão das predições, _precision_, _recall_, _F1 score_ e acurácia. Todas as métricas de predição devem ser calculadas a partir dos arquivos de treinamento por meio de validação cruzada.
O relatório deve ser enviado para o email do professor.

3 - **[2 pontos + equivalente a lista extra pela classificação]** A sua predição final do arquivo `test_X.csv` deve ser enviada para o professor por email. O formato deve ser o mesmo do arquivo `train_y.csv`, assim como exemplificado em `test_example_y.csv` (mas repare que as classes desse último arquivo foram gerados de forma aleatória). Em outras palavras, o arquivo a ser entregue deve ter dois campos. O campo `id` é o identificador do objeto em `test_X.csv` e o campo `label` é a classe que seu modelo encontrou para o objeto em questão. A primeira linha do arquivo deve conter os nomes das colunas.
A entrega desse arquivo é obrigatória e vale dois pontos. Além disso, o trabalho com maior _F1 score_ ganhará o equivalente a 100% de uma lista extra. O trabalho com o pior _F1 score_ não ganhará nota extra alguma. Os demais trabalhos terão nota proporcional.
O professor se reserva o direito de anular esse quesito (nota extra) se houver indícios de má conduta durante a competição.

**Kaggle:** Estou tentando criar uma competição para esse trabalho na plataforma _Kaggle_. Se eu conseguir, compartilho o _link_ com você no _Piazza_.

# Bibliotecas importadas

In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Leitura dos arquivos de treinamento e teste

In [37]:
df_train_values = pd.read_csv("data/train_X.csv")
df_train_labels = pd.read_csv("data/train_Y.csv")
ids = df_train_values["id"]

df_test_values = pd.read_csv("data/test_X.csv")

df_train_values.drop(columns=["id"])
df_train_labels.drop(columns=["id"])
df_test_values.drop(columns=["id"])

,a,b,c,d,e,f,g,h,i,j,k,l,m,n
0,19,2,267966,2,8,0,5,0,0,1,1,1,16,0
1,33,2,329433,3,15,0,0,4,0,1,1,1,41,0
2,48,4,123682,3,15,1,3,1,0,0,7689,1,61,0
3,37,2,201520,0,14,1,1,2,0,1,1,1,41,0
4,19,0,191118,2,8,0,4,3,0,0,1,1,21,0
5,49,4,118973,12,7,1,7,1,0,0,1,1,41,0
6,28,1,41100,1,10,0,6,0,0,0,1,1,31,0
7,36,3,105528,1,10,2,1,4,0,1,1,1,41,0
8,41,1,165816,5,11,0,6,3,0,0,1,1,41,0
9,42,2,230932,1,10,0,4,4,0,1,1,1,41,6


# Tratando arquivos

In [38]:
def categorical_attributes(df,cols):
    for char in cols:
        df = pd.concat(
            [df,pd.get_dummies(df[char], prefix=char)],axis=1)
        df.drop([char],axis=1, inplace=True)
    return df

cols = ["b", "d", "f", "g", "h", "i", "j", "n"]

df_train_values = categorical_attributes(df_train_values,cols)
df_test_values = categorical_attributes(df_test_values,cols)


In [12]:
# cValues = np.logspace(-10, 10, 100)
# clf = GridSearchCV(svm.SVC(kernel = 'linear'), param_grid = {'C': cValues}, n_jobs = -1)
# clf.fit(df_train_values, df_train_labels)
# print('Training done!')

In [ ]:
# df_answer_labels = clf.predict(df_train_values)
# print(accuracy_score(df_train_labels, df_answer_labels))
# cnf_matrix = confusion_matrix(y_test, y_pred)